用 MinimumEigenOptimizer 解决 qubo

In [1]:
from qiskit_optimization.problems import QuadraticProgram
qp = QuadraticProgram()
qp.binary_var('x')
qp.binary_var('y')
qp.binary_var('z')
qp.minimize(linear = {'y':-1}, quadratic = {('x','y'):2, ('z','y'):-4})
qp.linear_constraint(linear = {'x':1, 'y':2, 'z':3},
sense ="<=", rhs = 5)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - y + [ 4 x*y - 8 y*z ]/2
Subject To
 c0: x + 2 y + 3 z <= 5

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [2]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import NumPyMinimumEigensolver
np_solver = NumPyMinimumEigensolver()
np_optimizer = MinimumEigenOptimizer(np_solver)
result = np_optimizer.solve(qp)
print(result)

fval=-5.0, x=0.0, y=1.0, z=1.0, status=SUCCESS


C:\Users\Newt\AppData\Local\Temp\ipykernel_21068\3122343473.py:3: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  np_solver = NumPyMinimumEigensolver()


qaoa

In [3]:
from qiskit import Aer
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"),
shots = 1024)
qaoa = QAOA(optimizer = COBYLA(),
quantum_instance=quantum_instance, reps = 1)
qaoa_optimizer = MinimumEigenOptimizer(qaoa)
result = qaoa_optimizer.solve(qp)
print(result)

C:\Users\Newt\AppData\Local\Temp\ipykernel_21068\4112302810.py:5: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"),
C:\Users\Newt\AppData\Local\Temp\ipykernel_21068\4112302810.py:7: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(optimizer = COBYLA(),


fval=-5.0, x=0.0, y=1.0, z=1.0, status=SUCCESS


In [4]:
print('Variable order:', [var.name for var in result.variables])
for s in result.samples:
    print(s)

Variable order: ['x', 'y', 'z']
SolutionSample(x=array([0., 1., 1.]), fval=-5.0, probability=0.1396484375, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 1., 0.]), fval=-1.0, probability=0.0849609375, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 0., 1.]), fval=0.0, probability=0.1630859375, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 1.]), fval=0.0, probability=0.0107421875, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 0., 0.]), fval=0.0, probability=0.2978515625, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 0.]), fval=0.0, probability=0.1279296875, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1., 0.]), fval=1.0, probability=0.1162109375, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1., 1.]), fval=-3.0, probability=0.0595703125, status=<OptimizationResultStatus.INFEASIBLE: 2>)


In [5]:
print(result.min_eigen_solver_result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 30,
    'eigenstate': {   '000000': 0.05412658773652741,
                      '000011': 0.03125,
                      '000101': 0.13975424859373686,
                      '000110': 0.14320549046737,
                      '000111': 0.14320549046737,
                      '001000': 0.04419417382415922,
                      '001001': 0.125,
                      '001011': 0.07654655446197431,
                      '001101': 0.15934435979977452,
                      '001110': 0.184877493221863,
                      '001111': 0.12103072956898178,
                      '010000': 0.0625,
                      '010001': 0.21423920626253262,
                      '010010': 0.10825317547305482,
                      '010011': 0.1288470508005519,
                      '010100': 0.03125,
                      '010101': 0.15934435979977452,
                      '010110': 0.17399263633843817,
                      '010111': 0.076

In [6]:
from qiskit_optimization.converters import QuadraticProgramToQubo
qp_to_qubo = QuadraticProgramToQubo()
qubo = qp_to_qubo.convert(qp)
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 80 x - 161 y - 240 z - 80 c0@int_slack@0 - 160 c0@int_slack@1
      - 160 c0@int_slack@2 + [ 16 x^2 + 68 x*y + 96 x*z + 32 x*c0@int_slack@0
      + 64 x*c0@int_slack@1 + 64 x*c0@int_slack@2 + 64 y^2 + 184 y*z
      + 64 y*c0@int_slack@0 + 128 y*c0@int_slack@1 + 128 y*c0@int_slack@2
      + 144 z^2 + 96 z*c0@int_slack@0 + 192 z*c0@int_slack@1
      + 192 z*c0@int_slack@2 + 16 c0@int_slack@0^2
      + 64 c0@int_slack@0*c0@int_slack@1 + 64 c0@int_slack@0*c0@int_slack@2
      + 64 c0@int_slack@1^2 + 128 c0@int_slack@1*c0@int_slack@2
      + 64 c0@int_slack@2^2 ]/2 + 200
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1
 0 <= c0@int_slack@0 <= 1
 0 <= c0@int_slack@1 <= 1
 0 <= c0@int_slack@2 <= 1

Binaries
 x y z c0@int_slack@0 c0@int_slack@1 c0@int_slack@2
End



ising

In [7]:
H1, offset = qubo.to_ising()

print("The Hamiltonian is", H1)
print("The constant term is", offset)

The Hamiltonian is -7.0 * IIIIZI
- 4.5 * IIIIIZ
- 11.0 * IIIZII
- 4.0 * IIZIII
- 8.0 * IZIIII
- 8.0 * ZIIIII
+ 8.5 * IIIIZZ
+ 12.0 * IIIZIZ
+ 23.0 * IIIZZI
+ 4.0 * IIZIIZ
+ 8.0 * IIZIZI
+ 12.0 * IIZZII
+ 8.0 * IZIIIZ
+ 16.0 * IZIIZI
+ 24.0 * IZIZII
+ 8.0 * IZZIII
+ 8.0 * ZIIIIZ
+ 16.0 * ZIIIZI
+ 24.0 * ZIIZII
+ 8.0 * ZIZIII
+ 16.0 * ZZIIII
The constant term is 47.0


In [8]:
from qiskit_optimization.runtime import QAOAClient
from qiskit import IBMQ
provider = IBMQ.load_account()
qaoa_client = QAOAClient(provider=provider,backend=provider.get_backend("ibm_lagos"), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_client)
result = qaoa.solve(qp)
print(result)

C:\Users\Newt\AppData\Local\Temp\ipykernel_21068\575752498.py:3: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  provider = IBMQ.load_account()


RequestsApiError: '401 Client Error: Unauthorized for url: https://auth.quantum-computing.ibm.com/api/users/loginWithToken. Login failed., Error code: 3446.'